In [ ]:
!pip install mapbox

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 19.5 MB/s 
     |████████████████████████████████| 79 kB 10.7 MB/s 
     |████████████████████████████████| 8.9 MB 87.2 MB/s 
     |████████████████████████████████| 138 kB 83.1 MB/s 
     |████████████████████████████████| 127 kB 73.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
from requests.auth import HTTPBasicAuth
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from mapbox import Geocoder
import time, urllib.parse, math, random
from haversine import haversine, Unit, inverse_haversine
from sqlalchemy import create_engine

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/Education/Data Science/7. Data Engineering/'

In [ ]:
def fetch(ENDPOINT):
  api_url = f'https://webautomation.io/api/{ENDPOINT}'
  username = 'sam13a@aol.com'
  password = 'EYwz5UG%4%Nvw#M7Y*vPi#$bF7XVPWmUx^@N$%5'
  r = requests.post(api_url, auth=HTTPBasicAuth(f'{username}', f'{password}'), data={})
  r = requests.get(api_url, auth=HTTPBasicAuth(f'{username}', f'{password}'))
  return r

In [ ]:
r = fetch('sessions/data/211081')

In [ ]:
results = r.json()

In [ ]:
results.keys()

dict_keys(['id', 'spider', 'job_id', 'status', 'score', 'item_count', 'request_count', 'response_count', 'finish_reason', 'elapsed_time_seconds', 'results'])

In [ ]:
bedrooms = [results['results'][i]['bedrooms'] for i in range(len(results['results']))]

In [ ]:
rents = [results['results'][i]['price'] for i in range(len(results['results']))]

In [ ]:
addresses = [results['results'][i]['address'] for i in range(len(results['results']))]

In [ ]:
addresses_df = pd.DataFrame(addresses, columns=['address'])

In [ ]:
addresses_df['bedrooms'] = bedrooms
addresses_df['rent'] = rents

In [ ]:
addresses_df['bdrms'] = addresses_df['bedrooms'].str.split().str[0]

In [ ]:
addresses_df['rent_for_map'] = addresses_df['rent'].str.split('-').str[0].replace('[\$,]', '', regex=True).astype(int)

In [ ]:
addresses_df['bdrms'] = addresses_df['bdrms'].str.replace('Studio','studio').replace('1','1_br')

In [ ]:
addresses_df.to_pickle(path + 'addresses')

In [ ]:
addresses_df = pd.read_pickle(path + 'addresses')

In [ ]:
with open(path + 'mapbox_key.txt') as f:
  mapbox_access_token = f.read().rstrip()

In [ ]:
geocoder = Geocoder(access_token=mapbox_access_token)

In [ ]:
def get_geocoords(address):
  time.sleep(2)
  response = geocoder.forward(address)
  return response.json()['features'][0]['center']

In [ ]:
addresses_df['coords'] = addresses_df.apply(lambda x: get_geocoords(x.address), axis=1)

In [ ]:
addresses_df['lat'] = addresses_df['coords'].str[1]
addresses_df['long'] = addresses_df['coords'].str[0]
addresses_df.head()

,address,coords,lat,long
0,1788 N Pierce St Arlington VA 22209,"[-77.076558, 38.894855]",38.894855,-77.076558
1,700 N Randolph St Arlington VA 22203,"[-77.1094, 38.87927]",38.879270,-77.109400
2,950 S George Mason Dr Arlington VA 22204,"[-77.10242, 38.85923]",38.859230,-77.102420
3,3400 S Clark St Arlington VA 22202,"[-77.05215, 38.84472]",38.844720,-77.052150
4,320 23rd St S Arlington VA 22202,"[-77.051285, 38.852955]",38.852955,-77.051285


Generate random apartment data to supplement actual listings.

In [ ]:
def generate_map_data(workplace, distance, num_points=5000):
    def rent_gen(kind):
        studio_low = 1500
        one_br_low = 1700
        two_br_low = 2200
        high = 1000
        if kind == 'studio':
          return np.random.randint(studio_low, studio_low + high + 1)
        elif kind == '1_br': 
          return np.random.randint(one_br_low, one_br_low + high + 1)
        else:
          return np.random.randint(two_br_low, two_br_low + high + 1)

    map_data = [inverse_haversine(workplace, distance*math.sqrt(random.random()), random.random()*2*math.pi, unit=Unit.MILES) for _ in range(num_points)]
    map_data = pd.DataFrame(map_data, columns=['lat', 'long'])
    map_data['type'] = [random.choice(['studio', '1_br', '2_br']) for _ in range(map_data.shape[0])]


    map_data['rent'] = map_data['type'].apply(lambda x: rent_gen(x))

    return map_data

In [ ]:
def get_geocoords(address):
    workplace = urllib.parse.quote(address)
    try: url = f'https://api.mapbox.com/geocoding/v5/mapbox.places/{workplace}.json?access_token={mapbox_access_token}'
    except IndexError:
        print('Check your address again for typos. The address must be within the immediate DC/MD/VA area.')
        return
    session = requests.session()
    location = session.get(url)
    return location.json()['features'][0]['center']

In [ ]:
workplace = get_geocoords('732 N Kenmore St Arlington, VA 22201')[::-1]

In [ ]:
random_map_data = generate_map_data(workplace, 25, 5000)

In [ ]:
random_map_data['random_real'] = 'random'

In [ ]:
random_map_data.head()

,lat,long,type,rent,random_real
0,38.667336,-77.438078,studio,2213,random
1,38.855247,-77.468822,1_br,2119,random
2,39.136566,-76.915275,2_br,2939,random
3,38.994550,-76.686836,2_br,3154,random
4,38.831840,-76.698181,1_br,1972,random


In [ ]:
addresses_df['random_real'] = 'real'

In [ ]:
addresses_df_merge = addresses_df[['lat', 'long', 'bdrms', 'rent_for_map', 'random_real']].copy()

In [ ]:
addresses_df_merge.columns = random_map_data.columns

In [ ]:
map_data_arlington = pd.concat([addresses_df_merge, random_map_data]).reset_index(drop=True)

In [ ]:
map_data_arlington.to_pickle(path + 'map_data_arlington')

Create SQL database from this data.

In [ ]:
engine = create_engine(f'sqlite:////{path}/address_data_sql.db')


In [ ]:
# map_data_arlington.to_sql('address_data_sql', engine)
map_data_arlington.to_sql('address_data_table', engine)

In [ ]:
pd.read_sql('SELECT * FROM address_data_table;', engine)

,index,lat,long,type,rent,random_real
0,0,38.894855,-77.076558,1_br,3835,real
1,1,38.879270,-77.109400,studio,2000,real
2,2,38.859230,-77.102420,1_br,2058,real
3,3,38.844720,-77.052150,studio,1742,real
4,4,38.852955,-77.051285,studio,1744,real
...,...,...,...,...,...,...
5021,5021,38.908627,-76.895832,studio,1566,random
5022,5022,38.915288,-76.813617,studio,2344,random
5023,5023,39.079767,-76.753622,studio,2075,random
5024,5024,39.101965,-77.210269,1_br,1811,random
